In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pywsd
!pip install -U wn==0.0.23

     |████████████████████████████████| 26.8MB 115kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for pywsd: filename=pywsd-1.2.4-cp36-none-any.whl size=26940454 sha256=25a5540a13d950d6d0f7d1c2036f73c5913874b09c3f8f88c872a89581c0de8f
  Stored in directory: /root/.cache/pip/wheels/71/4d/d2/405b948047f7f3851f16ab9d893ce7c1a3010182900884536b
Successfully built pywsd
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
     |████████████████████████████████| 31.6MB 109kB/s 
  Created wheel for wn: filename=wn-0.0.23-cp36-none-any.whl size=31792944 sha256=eb79206d9719b93a28cd1c6fd5a993477f8da2a16a87f908102fb5c1fcea1f

In [ ]:
%cd /content/drive/MyDrive/task

/content/drive/MyDrive/task


In [ ]:
import os

import numpy as np
import nltk

from scipy import sparse
from tensorflow import keras

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from Datasets import SimilarityDataset
from models import SimilarityModel
from utility import top_k_metric

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train_ds = SimilarityDataset(mode="train", tag_func=nltk.pos_tag_sents)
val_ds = SimilarityDataset(mode="valid", tag_func=nltk.pos_tag_sents, input_tokenizer=train_ds.input_tokenizer, target_tokenizer=train_ds.target_tokenizer)
test_ds = SimilarityDataset(mode="test", tag_func=nltk.pos_tag_sents, input_tokenizer=train_ds.input_tokenizer, target_tokenizer=train_ds.target_tokenizer)

In [ ]:
train_data, train_labels = train_ds.get_data_target()
val_data, val_labels = val_ds.get_data_target()
test_data, test_labels = test_ds.get_data_target()

finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
finished setting up similarity matrix
finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)
finished loading corpus
finished loading descriptions
finished extracting contexts
finished extracting targets (OOCs)


## Save

In [ ]:
import json

input_tokenizer_json = train_ds.input_tokenizer.to_json()
with open("/content/drive/MyDrive/input_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(input_tokenizer_json, ensure_ascii=False))

target_tokenizer_json = train_ds.target_tokenizer.to_json()
with open("/content/drive/MyDrive/target_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(target_tokenizer_json, ensure_ascii=False))

# Load as:
# with open('tokenizer.json') as f:
#     data = json.load(f)
#     tokenizer = tokenizer_from_json(data)

sparse.save_npz("/content/drive/MyDrive/new_vocab_sim_mat", train_ds.sim_mat)

# Load as:
# sim_mat = sparse.load_npz("/content/drive/MyDrive/sim_mat")

## Model

In [ ]:
from importlib import reload
import models
reload(models)
reload(models.similarity)
from models.similarity import SimilarityModel

In [ ]:
model = SimilarityModel()
model.build_model(
        len(train_ds.input_tokenizer.word_index) + 1,
        train_labels.shape[-1],
        (train_data.shape[1],),
        train_ds.sim_mat.toarray(),
        metrics=[
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
        ],
        hidden_sizes=[256, 512])

In [ ]:
model.train(train_data, train_labels, val_data, val_labels, batch_size=16, epochs=10)

Epoch 1/10
539/539 [==============================] - 12s 17ms/step - loss: 0.4500 - precision: 0.0010 - recall: 0.2330 - val_loss: 0.0103 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
539/539 [==============================] - 9s 17ms/step - loss: 0.0099 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0083 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
539/539 [==============================] - 9s 16ms/step - loss: 0.0087 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0080 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
539/539 [==============================] - 9s 17ms/step - loss: 0.0084 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0079 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
539/539 [==============================] - 9s 17ms/step - loss: 0.0082 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/10
5

## Predictions

In [ ]:
train_pred = model.infer(train_data)
val_pred = model.infer(val_data)
test_pred = model.infer(test_data)

In [ ]:
for i in range(1, 11):
    metric = top_k_metric(train_pred, train_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.06049698095680446
recall: 0.027476334240152167
f1: 0.03778953346404715
top 2: 
precision: 0.06009057129586623
recall: 0.05619429146484418
f1: 0.058077156347297515
top 3: 
precision: 0.05562006502554574
recall: 0.07899000287527924
f1: 0.06527638184579136
top 4: 
precision: 0.05059800278680911
recall: 0.0963078926414969
f1: 0.06634161285439114
top 5: 
precision: 0.04821179749187181
recall: 0.11594618804326189
f1: 0.06810480915288575
top 6: 
precision: 0.04530500077411363
recall: 0.13202452834361797
f1: 0.06746052266160908
top 7: 
precision: 0.043593656691659474
recall: 0.14842993165682436
f1: 0.06739383988253826
top 8: 
precision: 0.04156990246168137
recall: 0.16150194634286597
f1: 0.06612063854607798
top 9: 
precision: 0.0393894823760128
recall: 0.17192151593568222
f1: 0.06409415104856062
top 10: 
precision: 0.037563864375290294
recall: 0.18118489151350275
f1: 0.062226682515396085


In [ ]:
for i in range(1, 11):
    metric = top_k_metric(val_pred, val_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.04331450094161959
recall: 0.021249215317011922
f1: 0.028511343838090866
top 2: 
precision: 0.05084745762711865
recall: 0.052589453860640296
f1: 0.051703787136542126
top 3: 
precision: 0.04959196484620213
recall: 0.07614563716258631
f1: 0.060064955924556344
top 4: 
precision: 0.047080979284369114
recall: 0.09511927181418707
f1: 0.06298594315032804
top 5: 
precision: 0.045574387947269306
recall: 0.11382611424984306
f1: 0.06508832052408078
top 6: 
precision: 0.04378531073446327
recall: 0.129504080351538
f1: 0.06544401090034249
top 7: 
precision: 0.04210384718859295
recall: 0.14309478970495917
f1: 0.06506355835312529
top 8: 
precision: 0.040725047080979286
recall: 0.15886691776522285
f1: 0.0648308934739351
top 9: 
precision: 0.03902490060682151
recall: 0.17231638418079095
f1: 0.06363763494993567
top 10: 
precision: 0.03728813559322035
recall: 0.1810353331539772
f1: 0.06183916084281225


In [ ]:
for i in range(1, 11):
    metric = top_k_metric(test_pred, test_labels, i)
    print(f"top {i}: \nprecision: {metric[0]}\nrecall: {metric[1]}\nf1: {metric[2]}")

top 1: 
precision: 0.051789077212806026
recall: 0.024026993094789704
f1: 0.03282511993906273
top 2: 
precision: 0.05555555555555555
recall: 0.05473948524795982
f1: 0.055144501359640795
top 3: 
precision: 0.05053358443188951
recall: 0.07725988700564973
f1: 0.061102010600071734
top 4: 
precision: 0.049199623352165725
recall: 0.10054927809165097
f1: 0.06607042272423513
top 5: 
precision: 0.04821092278719397
recall: 0.1229127432517263
f1: 0.06925677682855892
top 6: 
precision: 0.04519774011299435
recall: 0.1346829880728186
f1: 0.06768225539167991
top 7: 
precision: 0.04331450094161958
recall: 0.15062774639045823
f1: 0.06728153099816551
top 8: 
precision: 0.04084274952919021
recall: 0.16285310734463276
f1: 0.06530686264716003
top 9: 
precision: 0.038606403013182675
recall: 0.17330508474576273
f1: 0.0631460428755945
top 10: 
precision: 0.03766478342749529
recall: 0.18931261770244823
f1: 0.06282932759259845
